In [1]:
import pandas as pd
import numpy as np

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [32]:
import random
l=[random.randint(0,10) for i in range(10)]
#1. 
print('First question: \n')
file_name = "data_int.txt"
with open(file_name, mode = 'w') as f:
    f.write(str(l))
!cat data_int.txt
print('\n\nSecond question \n')
#2. 
mat=[[random.uniform(0,10) for i in range(5)] for j in range(5)]
file_name2 = "data_float.txt"
np.savetxt('data_float.txt', mat)
!cat data_float.txt
print('\n\nThird question \n')
read_file = pd.read_csv('data_float.txt', nrows=5, skiprows=range(1,1))
read_file.to_csv (r'data_float.csv', index=None)
print(read_file)

First question: 

[1, 3, 5, 2, 0, 3, 8, 1, 5, 2]

Second question 

1.109163309653760088e-01 3.658470006223828452e+00 2.996264337963150393e-01 2.726650990307900813e+00 5.868905722124047486e+00
1.346962077115241296e+00 1.855232854372896867e+00 2.617486715499797167e+00 7.449837546485422735e+00 1.620626981144679402e-01
6.891092763051331005e+00 8.565824817930645096e-01 6.706545528523655264e-01 8.620695649004476380e+00 2.968848551641114497e+00
2.863627317010851492e+00 1.707314950177686308e+00 8.071849657745195472e+00 4.124273465520373882e+00 1.820005443189364192e+00
5.492850131147687165e+00 7.761563693912244766e+00 6.468071518958424804e+00 6.349502267008349854e+00 5.153104603434751851e+00


Third question 

  1.109163309653760088e-01 3.658470006223828452e+00 2.996264337963150393e-01 2.726650990307900813e+00 5.868905722124047486e+00
0  1.346962077115241296e+00 1.855232854372896867e...                                                                          
1  6.891092763051331005e+00 8.5658

2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [39]:
#from urllib.request import urlopen
#import json

#data = json.loads(urlopen('https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json').read().decode("utf-8"))

import requests, json

content = requests.get("https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json")
json = json.loads(content.content)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?